In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
# check apakah GPU tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
# === TAHAP 1: PERSIAPAN DATA ===
# Transformasi gambar: resize, augmentasi, normalisasi
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# dataset custom untuk gender
class GenderDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # label: 0 untuk female, 1 untuk male
        for label, gender in enumerate(['female', 'male']):
            gender_path = os.path.join(self.root_dir, gender)
            for img_name in os.listdir(gender_path):
                self.image_paths.append(os.path.join(gender_path, img_name))
                self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except:
            # jika gambar rusak, return gambar dummy
            print(f"Warning: Could not load image {img_path}. Skipping.")
            return self.__getitem__((idx + 1) % len(self))
            
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

# dataset custom untuk usia
class AgeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # loop melalui setiap folder di direktori root
        for age_folder in os.listdir(self.root_dir):
            folder_path = os.path.join(self.root_dir, age_folder)
            
            if os.path.isdir(folder_path):
                try:
                    # coba pisahkan nama folder berdasarkan tanda '-'
                    parts = age_folder.split('-')
                    if len(parts) == 2:
                        # jika berhasil (misal: "18-20"), hitung rata-ratanya
                        start_age = int(parts[0])
                        end_age = int(parts[1])
                        avg_age = (start_age + end_age) / 2.0
                    else:
                        # jika formatnya angka tunggal (misal: "18")
                        avg_age = float(age_folder)

                    # jika berhasil, tambahkan semua gambar di folder itu
                    for img_name in os.listdir(folder_path):
                        self.image_paths.append(os.path.join(folder_path, img_name))
                        self.labels.append(avg_age)
                        
                except ValueError:
                    # abaikan folder yang namanya tidak bisa diubah jadi angka (misal: '.DS_Store')
                    print(f"Mengabaikan folder: {age_folder}")
                    continue

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
        except:
            print(f"Warning: Could not load image {img_path}. Skipping.")
            return self.__getitem__((idx + 1) % len(self))
        
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.float32)

# Ganti dengan path dataset Anda
GENDER_DATA_PATH = './dataset/gender/Training/'
AGE_DATA_PATH = './dataset/age/Training/'

# Buat instance dataset
# CATATAN: Untuk kesederhanaan, kita gunakan path training untuk validasi juga.
# Dalam proyek nyata, Anda harus memiliki folder validasi terpisah.
gender_dataset_train = GenderDataset(root_dir=GENDER_DATA_PATH, transform=data_transforms['train'])
age_dataset_train = AgeDataset(root_dir=AGE_DATA_PATH, transform=data_transforms['train'])

gender_dataset_val = GenderDataset(root_dir=GENDER_DATA_PATH, transform=data_transforms['val'])
age_dataset_val = AgeDataset(root_dir=AGE_DATA_PATH, transform=data_transforms['val'])

print(f"path yang diperiksa untuk gender: {os.path.abspath(GENDER_DATA_PATH)}")
print(f"jumlah gambar gender yang ditemukan: {len(gender_dataset_train)}")
print(f"path yang diperiksa untuk usia: {os.path.abspath(AGE_DATA_PATH)}")
print(f"jumlah gambar usia yang ditemukan: {len(age_dataset_train)}")

# DataLoader
BATCH_SIZE = 32
gender_loader_train = DataLoader(gender_dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
age_loader_train = DataLoader(age_dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

gender_loader_val = DataLoader(gender_dataset_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
age_loader_val = DataLoader(age_dataset_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)


print(f"""
Gender dataset size: {len(gender_dataset_train)}
Age dataset size: {len(age_dataset_train)}""")

path yang diperiksa untuk gender: c:\DeepLearning\agegenderclassification-project\dataset\gender\Training
jumlah gambar gender yang ditemukan: 220
path yang diperiksa untuk usia: c:\DeepLearning\agegenderclassification-project\dataset\age\Training
jumlah gambar usia yang ditemukan: 125

Gender dataset size: 220
Age dataset size: 125


In [4]:
# === TAHAP 2: DEFINISI MODEL ===

class AgeGenderModel(nn.Module):
    def __init__(self, num_features):
        super(AgeGenderModel, self).__init__()
        # pake ResNet50 sebagai base model
        self.base_model = models.resnet50(pretrained=True)
        
        # "Bekukan" bobot dari base model agar tidak ikut terlatih
        for param in self.base_model.parameters():
            param.requires_grad = False
            
        # ganti lapisan klasifikasi asli ResNet50
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity() # hapus lapisan fc asli

        # gender branch
        self.gender_head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1) # output tunggal untuk sigmoid
        )
        
        # Age Branch
        self.age_head = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1) # Output tunggal untuk prediksi usia
        )

    def forward(self, x):
        features = self.base_model(x)
        features = features.view(features.size(0), -1)
        
        gender_output = self.gender_head(features)
        age_output = self.age_head(features)
        
        return gender_output, age_output

# inisialisasi model
model = AgeGenderModel(num_features=2048)
model = model.to(device)

c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# === STEP 3: TRAINING & VALIDATION ===
# loss functions and optimizer
criterion_gender = nn.BCEWithLogitsLoss() # lebih stabil dari Sigmoid + BCELoss
criterion_age = nn.L1Loss() # Mean Absolute Error (MAE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

NUM_EPOCHS = 20

def train_model(model, gender_loader, age_loader, criterion_gender, criterion_age, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        running_gender_loss = 0.0
        running_age_loss = 0.0
        
        # gunakan iterator untuk mengambil data secara bergantian
        age_iter = iter(age_loader)
        
        progress_bar = tqdm(gender_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        
        for i, (gender_inputs, gender_labels) in enumerate(progress_bar):
            # --- Training Gender ---
            gender_inputs = gender_inputs.to(device)
            gender_labels = gender_labels.to(device).unsqueeze(1)
            
            optimizer.zero_grad()
            
            gender_outputs, _ = model(gender_inputs)
            loss_gender = criterion_gender(gender_outputs, gender_labels)
            
            # --- Training Age ---
            try:
                age_inputs, age_labels = next(age_iter)
            except StopIteration:
                # jika data usia habis, reset iterator
                age_iter = iter(age_loader)
                age_inputs, age_labels = next(age_iter)

            age_inputs = age_inputs.to(device)
            age_labels = age_labels.to(device).unsqueeze(1)

            _, age_outputs = model(age_inputs)
            loss_age = criterion_age(age_outputs, age_labels)

            # gabungkan loss dan lakukan backpropagation
            total_loss = loss_gender + loss_age
            total_loss.backward()
            optimizer.step()
            
            running_gender_loss += loss_gender.item() * gender_inputs.size(0)
            running_age_loss += loss_age.item() * age_inputs.size(0)
            
            progress_bar.set_postfix(gender_loss=f"{loss_gender.item():.4f}", age_loss=f"{loss_age.item():.4f}")

        epoch_gender_loss = running_gender_loss / len(gender_loader.dataset)
        epoch_age_loss = running_age_loss / len(age_loader.dataset)
        
        print(f"Epoch {epoch+1}/{num_epochs} -> Train Gender Loss: {epoch_gender_loss:.4f}, Train Age MAE: {epoch_age_loss:.4f}")
        
        # --- Validasi ---
        validate_model(model, gender_loader_val, age_loader_val, criterion_gender, criterion_age)
        
    print("Training selesai!")
    return model

def validate_model(model, gender_loader, age_loader, criterion_gender, criterion_age):
    model.eval()
    total_gender_loss = 0
    total_age_loss = 0
    gender_corrects = 0
    
    with torch.no_grad():
        # validasi gender
        for inputs, labels in gender_loader:
            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
            outputs, _ = model(inputs)
            loss = criterion_gender(outputs, labels)
            total_gender_loss += loss.item() * inputs.size(0)
            
            preds = torch.sigmoid(outputs) > 0.5
            gender_corrects += torch.sum(preds == labels.data)

        # validasi age
        for inputs, labels in age_loader:
            inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
            _, outputs = model(inputs)
            loss = criterion_age(outputs, labels)
            total_age_loss += loss.item() * inputs.size(0)
            
    avg_gender_loss = total_gender_loss / len(gender_loader.dataset)
    avg_age_mae = total_age_loss / len(age_loader.dataset)
    gender_acc = gender_corrects.double() / len(gender_loader.dataset)
    
    print(f"Validation -> Gender Loss: {avg_gender_loss:.4f}, Gender Acc: {gender_acc:.4f}, Age MAE: {avg_age_mae:.4f}\n")


In [6]:
# === STEP 4: EKSEKUSI ===
trained_model = train_model(model,
                            gender_loader_train,
                            age_loader_train,
                            criterion_gender,
                            criterion_age,
                            optimizer,
                            num_epochs=NUM_EPOCHS)

# save the trained model
torch.save(trained_model.state_dict(), 'age_gender_model.pth')
print("The model has been saved as age_gender_model.pth")


# === INFERENCE ===
def predict_image(image_path, model):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    
    transform = data_transforms['val']
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        gender_output, age_output = model(image_tensor)
        
        gender_prob = torch.sigmoid(gender_output).item()
        gender = "Male" if gender_prob > 0.5 else "Female"
        
        age = age_output.item()
        
    print(f"Prediction for {image_path}:")
    print(f"  - Gender: {gender} (Prob: {gender_prob:.2f})")
    print(f"  - Age: {age:.1f} years old")

# Coba prediksi pada gambar baru (ganti dengan path gambar Anda)
# predict_image('path/to/your/test_image.jpg', trained_model)

Epoch 1/20: 100%|██████████| 7/7 [00:57<00:00,  8.27s/it, age_loss=14.4976, gender_loss=0.7404]


Epoch 1/20 -> Train Gender Loss: 0.8409, Train Age MAE: 38.8405
Validation -> Gender Loss: 0.6029, Gender Acc: 0.7455, Age MAE: 11.7876



Epoch 2/20: 100%|██████████| 7/7 [01:07<00:00,  9.62s/it, age_loss=16.1560, gender_loss=0.6420]


Epoch 2/20 -> Train Gender Loss: 0.6135, Train Age MAE: 27.6103
Validation -> Gender Loss: 0.5185, Gender Acc: 0.8500, Age MAE: 12.8188



Epoch 3/20: 100%|██████████| 7/7 [01:07<00:00,  9.63s/it, age_loss=12.4078, gender_loss=0.4958]


...

Epoch 18/20 -> Train Gender Loss: 0.3367, Train Age MAE: 14.2212
Validation -> Gender Loss: 0.2034, Gender Acc: 0.9364, Age MAE: 7.4099



Epoch 19/20: 100%|██████████| 7/7 [00:53<00:00,  7.71s/it, age_loss=6.2467, gender_loss=0.2063]


Epoch 19/20 -> Train Gender Loss: 0.2779, Train Age MAE: 13.2552
Validation -> Gender Loss: 0.3033, Gender Acc: 0.8682, Age MAE: 7.7070



Epoch 20/20: 100%|██████████| 7/7 [00:51<00:00,  7.42s/it, age_loss=9.0552, gender_loss=0.2884]


Epoch 20/20 -> Train Gender Loss: 0.2607, Train Age MAE: 14.0849
Validation -> Gender Loss: 0.2417, Gender Acc: 0.9182, Age MAE: 7.1064

Training selesai!
The model has been saved as age_gender_model.pth


In [8]:
# === TAMBAHAN: DETEKSI LIVE DARI KAMERA ===

import cv2
from PIL import Image
from IPython.display import display, clear_output

# Pastikan class AgeGenderModel sudah terdefinisi di sel sebelumnya.

# === 1. Muat Model yang Sudah Dilatih ===
print("Memuat model yang sudah dilatih...")
device = torch.device("cpu") # Gunakan CPU untuk inferensi agar lebih mudah
live_model = AgeGenderModel(num_features=2048).to(device)
live_model.load_state_dict(torch.load('age_gender_model.pth', map_location=device))
live_model.eval()
print("Model berhasil dimuat.")


# === 2. Muat Face Detector dari OpenCV ===
# OpenCV menyediakan model deteksi wajah Haar Cascade yang sudah terlatih.
# Pastikan file haarcascade_frontalface_default.xml ada.
# Jika error, unduh file tersebut dan letakkan di folder yang sama dengan notebook ini.
try:
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    print("Face detector berhasil dimuat.")
except Exception as e:
    print(f"Error memuat face detector: {e}")
    print("Pastikan OpenCV terinstal dengan benar atau path ke file XML benar.")


# === 3. Loop untuk Deteksi Live ===
# Transformasi gambar input (sama seperti validasi)
transform = data_transforms['val']

# Buka koneksi ke webcam (0 biasanya untuk webcam internal)
video_capture = cv2.VideoCapture(0)
print("Kamera dinyalakan. Tekan 'q' di jendela output untuk berhenti.")

try:
    while True:
        # Baca setiap frame dari video
        ret, frame = video_capture.read()
        if not ret:
            print("Gagal mengambil frame dari kamera.")
            break

        # Konversi frame ke grayscale untuk deteksi wajah (lebih cepat)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Deteksi wajah dalam gambar grayscale
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(100, 100) # Ukuran minimum wajah yang akan dideteksi
        )

        # Loop untuk setiap wajah yang terdeteksi
        for (x, y, w, h) in faces:
            # Gambar kotak di sekeliling wajah pada frame asli (berwarna)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Crop area wajah dari frame
            face_roi = frame[y:y+h, x:x+w]

            # Konversi ROI (NumPy array) ke PIL Image untuk transformasi
            pil_image = Image.fromarray(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))
            
            # Terapkan transformasi
            image_tensor = transform(pil_image).unsqueeze(0).to(device)
            
            # Lakukan prediksi dengan model
            with torch.no_grad():
                gender_output, age_output = live_model(image_tensor)
                
                gender_prob = torch.sigmoid(gender_output).item()
                gender = "Pria" if gender_prob > 0.5 else "Wanita"
                age = age_output.item()
            
            # Siapkan teks untuk ditampilkan
            prediction_text = f"{gender}, Usia: {age:.1f}"
            
            # Tampilkan teks prediksi di atas kotak wajah
            cv2.putText(frame, prediction_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Tampilkan frame yang sudah di-update di output Jupyter
        # cv2.imshow seringkali tidak bekerja baik di notebook. Cara ini lebih stabil.
        _, jpg_frame = cv2.imencode('.jpeg', frame)
        display(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
        
        # Hapus output sebelumnya agar video terlihat seperti streaming
        clear_output(wait=True)
        
        # Cek jika tombol 'q' ditekan untuk keluar (mungkin tidak berfungsi di semua browser)
        # Cara terbaik untuk stop adalah dengan menekan tombol Stop di Jupyter Notebook.

except KeyboardInterrupt:
    print("Streaming dihentikan oleh pengguna.")

finally:
    # Lepaskan koneksi kamera dan tutup semua jendela
    video_capture.release()
    print("Kamera sudah dimatikan.")

Streaming dihentikan oleh pengguna.
Kamera sudah dimatikan.
